In [3]:
import os
os.chdir(r"C:\Users\admin\Downloads\Code\ObjectDetection")

categories = []
with open("yolov5/data/smartcity.yaml", 'r') as f:
    lines = f.readlines()
for line in lines:
    if line.strip().startswith("names:"):
        name_line = eval(line.strip().split("names:")[1].strip())
        for i, name in enumerate(name_line):
            categories.append({"id": i, "name": name})


脚本：YOLO ➜ COCO JSON for Faster R-CNN

In [5]:
import os
import shutil
import json
from tqdm import tqdm

# 输入路径
yolo_root = "../coco_subset"
output_root = "../coco_subset_faster"
os.makedirs(output_root, exist_ok=True)

def convert_split(split):
    print(f"Converting {split} set...")
    
    # 创建新结构
    image_dir = os.path.join(output_root, "images", split)
    ann_dir = os.path.join(output_root, "annotations")
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(ann_dir, exist_ok=True)

    label_dir = os.path.join(yolo_root, "labels", split)
    source_image_dir = os.path.join(yolo_root, "images", split)

    # 类别信息（从 YOLOv5 的 smartcity.yaml 复制）
    # 类别信息：直接写入 COCO 80类标签
    categories = [
    {"id": i, "name": name} for i, name in enumerate([
        'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck',
        'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
        'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
        'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
        'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
        'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse',
        'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
        'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
        'toothbrush'
        ])
    ]


    images = []
    annotations = []
    ann_id = 1
    for idx, file_name in enumerate(tqdm(os.listdir(label_dir))):
        if not file_name.endswith(".txt"):
            continue
        img_name = file_name.replace(".txt", ".jpg")
        src_img_path = os.path.join(source_image_dir, img_name)
        dst_img_path = os.path.join(image_dir, img_name)
        shutil.copyfile(src_img_path, dst_img_path)

        height, width = 640, 640  # 如果知道原始尺寸可以替换

        image_info = {
            "id": idx,
            "file_name": img_name,
            "width": width,
            "height": height
        }
        images.append(image_info)

        # 读取 YOLO 标签
        label_path = os.path.join(label_dir, file_name)
        with open(label_path, 'r') as f:
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                class_id = int(parts[0])
                x_center, y_center, w, h = map(float, parts[1:])
                x = x_center - w / 2
                y = y_center - h / 2

                ann = {
                    "id": ann_id,
                    "image_id": idx,
                    "category_id": class_id,
                    "bbox": [x * width, y * height, w * width, h * height],
                    "area": w * h * width * height,
                    "iscrowd": 0
                }
                annotations.append(ann)
                ann_id += 1

    coco_json = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    output_path = os.path.join(ann_dir, f"annotations_{split}.json")
    with open(output_path, 'w') as f:
        json.dump(coco_json, f)

    print(f"{split} annotations saved to {output_path}")

# 执行两个 split
convert_split("train")
convert_split("val")


Converting train set...


FileNotFoundError: [Errno 2] No such file or directory: '..\\yolov5\\data\\smartcity.yaml'

Step 1：环境配置与导入库

In [ ]:
import os
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision.datasets import CocoDetection
import matplotlib.pyplot as plt
